In [ ]:
# based on https://towardsdatascience.com/artificial-neural-networks-optimization-using-genetic-algorithm-with-python-1fe8ed17733e
from common import Trial, safe_log, nll, load_df, df_to_ML_data, timed_method

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import numpy as np
import pickle
import keras
from keras import backend as K
from keras.models import model_from_json
from keras.layers import Dense, Flatten, BatchNormalization, Dropout, Lambda
from keras.layers import Conv2D, AveragePooling2D
from keras.models import Sequential
import keras_metrics
from keras import metrics
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
import matplotlib
import matplotlib.pyplot
import logging
import random
import sys
logging.basicConfig(format='%(asctime)s | %(levelname)s : %(message)s',level=logging.INFO, 
                    filename='genetic_learning_weights_only_coarse_mix.log', filemode='w+')


log = logging.getLogger(__name__)



config = tf.ConfigProto()#device_count = {'GPU': 0})
config.gpu_options.per_process_gpu_memory_fraction = 0.2
config.gpu_options.allow_growth = True
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))


In [ ]:
def vectorize_population(population_of_models):
    population_vector = []
    for model in population_of_models:
        model_vector = []
        for layer in model:
            layer_vectorized = np.reshape(layer, newshape=(layer.size))
            np.reshape(layer_vectorized, newshape=layer.shape)
            model_vector.extend(layer_vectorized)
        population_vector.append(model_vector)
    return np.array(population_vector)

def population_vector_to_models(population_vectors, reference_model):
    models = []
    model_idx = 0
    for vector in population_vectors:
        start = 0
        end = 0
        model = []
        for reference_layer in reference_model:
            end = end + reference_layer.size
            layer_vector = vector[start:end]
            layer = np.reshape(layer_vector, newshape=reference_layer.shape)
            model.append(layer) 
            start = end
        models.append(model)
        model_idx = model_idx + 1

    return models

In [ ]:
with open('json_weights_bn_simple.pkl', 'rb') as f:
    model_topology_json, weights = pickle.load(f)

(model_topology_json, weights)

In [ ]:
X_train, X_test, y_train, y_test = df_to_ML_data(load_df())

In [ ]:
def check_weights(a, b):
    assert len(a) == len(b)
    for i in range(len(a)):
        assert a[i].shape ==  b[i].shape

@timed_method
def create_model_programatically(weights = None):
    input_shape = (64, 256, 1)
    num_classes = 2
    
    model = Sequential()
    model.add(Conv2D(30, kernel_size=(1, 25),
                     input_shape=input_shape))
    model.add(BatchNormalization(momentum=0.1))
    model.add(Conv2D(10, kernel_size=(64, 1)))
    model.add(BatchNormalization(momentum=0.1))
    model.add(Lambda(lambda x: x ** 2))
    model.add(AveragePooling2D(pool_size=(1, 15), strides=(1, 1)))
    model.add(Lambda(lambda x: safe_log(x)))
    model.add(BatchNormalization(momentum=0.1))
    #model.add(Dropout(0.5))
    model.add(Conv2D(2, kernel_size=(1, 8), dilation_rate=(15, 1)))
    model.add(BatchNormalization(momentum=0.1))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax'))


    if weights != None:
        check_weights(model.get_weights(), weights)

        model.set_weights(weights)

        
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True),
              metrics=[keras_metrics.binary_f1_score(), metrics.binary_accuracy])
                        #metrics.binary_accuracy, metrics.cosine_proximity, metrics.mean_absolute_error, 
                       #metrics.mean_absolute_percentage_error, metrics.mean_squared_error,
                       #keras_metrics.precision(), keras_metrics.recall(), keras_metrics.binary_f1_score(), 
                       #keras_metrics.binary_false_negative(), keras_metrics.binary_false_positive(),
                       #keras_metrics.binary_true_negative(), keras_metrics.binary_true_positive()
                        #])

    
    return model

@timed_method
def individual_fitness(model_topology_json, model_weights, X, y):
    #keras_model = model_from_json(model_topology_json)
    keras_model = create_model_programatically(model_weights)
    result = keras_model.evaluate(X,y, verbose=0) 
    log.debug(result)
    return result[1]
    

def population_fitness(model_topology_json, population_of_models, X, y):
    fitness_scores = []
    
    for model_weights in population_of_models:#tqdm(population_of_models, desc='Current Population Fitness', position=1):
        fitness_scores.append(individual_fitness(model_topology_json, model_weights, X, y))
                          
    return fitness_scores

def fittest_parents_of_generation(population, fitness_scores, num_parents):
    # Selecting the best individuals in the current generation as parents for producing the offspring of the next generation.
    parents = []
    for _ in range(num_parents - 1):
        max_fitness_idx = np.argmax(fitness_scores)
        parents.append(population[max_fitness_idx])
        del population[max_fitness_idx]
        del fitness_scores[max_fitness_idx]
    
    # mixing in a lucky one.. because sometimes anyone can get lucky ;)
    np.random.shuffle(population)
    parents.append(population[0])

    return parents

#crossover that produces the offspring by taking the first half of it from the first parent 
# and taking the second half of it from the 2nd parent
def crossover(parent_vectors, offsprings_size):
    offsprings = np.empty((offsprings_size, len(parent_vectors[0])))
    # The point at which crossover takes place between two parents. Usually, it is at the center.
    crossover_point = int(len(parent_vectors[0]) / 2)
    for k in range(offsprings_size):
        # Index of the first parent to mate.
        parent1_idx = k % len(parent_vectors)
        # Index of the second parent to mate.
        parent2_idx = (k+1) % len(parent_vectors)
        # The new offspring will have its first half of its genes taken from the first parent.
        offsprings[k, 0:crossover_point] = parent_vectors[parent1_idx][0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offsprings[k, crossover_point:] = parent_vectors[parent2_idx][crossover_point:]
    return offsprings


def mutation(offsprings, mutation_chance=0.1, mutation_rate=1):
    num_mutations = np.uint8(len(offsprings[0]) * mutation_chance)
    mutation_indices = np.array(random.sample(range(0, len(offsprings[0])), num_mutations))
    log.debug('Mutating indices = %s', mutation_indices)

    # Mutation changes a single gene in each offspring randomly.
    for idx in range(len(offsprings)):
        # The random value to be added to the gene.
        mutation_value = np.random.uniform(-1.0 * mutation_rate, 1.0 * mutation_rate, 1)
        offsprings[idx][mutation_indices] = offsprings[idx][mutation_indices] + mutation_value
        
    return offsprings

In [ ]:
#population_fitness(model_topology_json, [weights], X_train, y_train)
#for i in range(100):
individual_fitness(None, weights, X_test, y_test)
#    if i % 5 == 0:
#        K.clear_session()

In [ ]:
#Genetic algorithm parameters:
#    Mating Pool Size (Number of Parents)
#    Population Size
#    Number of Generations
#    Mutation Percent


population_size = 20 #sol_per_pop
num_parents_mating = 8
num_generations = 1000
mutation_chance = 0.1
mutation_rate = 2
reference_weights = weights

#Creating the initial population.
population_weights = []
for _ in range(0, population_size):
    population_weights.append(create_model_programatically().get_weights())
    
K.clear_session()

In [ ]:
individual_fitness(None, population_weights[2], X_test, y_test)
individual_fitness(None, population_weights[3], X_test, y_test)

In [ ]:
best_of_each_generation = []

for generation in tqdm(range(num_generations), desc='Generations', position=0):

    log.debug('Testing generation |%d| population: |%s|', generation, population_weights)
    # Measuring the fitness of each chromosome in the population.
    fitness_scores = population_fitness(None, population_weights, X_test, y_test)
    
    best_of_this_generation = max(fitness_scores)
    best_of_each_generation.append(best_of_this_generation)
    log.info("Best of geration |%d| has accuracy of |%f|", generation, best_of_this_generation)
    log.info('Fitness scores of this generation: |%s|', fitness_scores)
    
    
    # Selecting the best parents in the population for mating.
    parents = fittest_parents_of_generation(population_weights.copy(), fitness_scores, num_parents_mating)

    # Generating next generation using crossover.
    offsprings = crossover(vectorize_population(parents), len(population_weights) - num_parents_mating)

    # Adding some variations to the offsrping using mutation.
    offsprings = mutation(offsprings, mutation_chance=mutation_chance, mutation_rate=mutation_rate)

    # Creating the new generation based on the parents and offspring.
    population_weights = []
    population_weights.extend(parents)
    population_weights.extend(population_vector_to_models(offsprings, parents[0]))

    #cleanup resources
    K.clear_session()
    



In [ ]:
fitness_scores = population_fitness(None, population_weights, X_test, y_test)
log.info("Accuracy of the best solution is : ", max(fitness_scores))
log.info('Fitness scores of the last generation: |%s|', fitness_scores)


In [ ]:
matplotlib.pyplot.plot(best_of_each_generation, linewidth=5, color="black")
matplotlib.pyplot.xlabel("Iteration", fontsize=20)
matplotlib.pyplot.ylabel("Fitness", fontsize=20)
#matplotlib.pyplot.xticks(np.arange(0, num_generations+1, 100), fontsize=15)
#matplotlib.pyplot.yticks(np.arange(0, 101, 5), fontsize=15)


In [ ]:
import IPython
IPython.display.Audio("F:\\Tresorit\\01 - Startup Screen.mp3", autoplay=True)
